In [3]:
pip pwd

Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "pwd"



In [1]:
import pandas as pd
import os
import pwd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ModuleNotFoundError: No module named 'pwd'

delivery_data.isnull().sum()
ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Type_of_order                  0
Type_of_vehicle                0
Time_taken(min)                0
Distance_km                    0
dtype: int64

In [ ]:
delivery_data = pd.read_csv("deliverytime.csv")
delivery_data.head()


We have calculated the distances between the restaurant and delivery locations which are based on the coordinates , The distance is important column for predicting the delivery time . Over here we have used Haversine formula to find the distances   betwen  2 points on earth surface given with their latitudes and longitudes 

In [ ]:
from math import radians, cos, sin, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Calculate distance for each row in the DataFrame
delivery_data['Distance_km'] = delivery_data.apply(lambda row: haversine(row['Restaurant_latitude'], 
                                                                         row['Restaurant_longitude'], 
                                                                         row['Delivery_location_latitude'], 
                                                                         row['Delivery_location_longitude']), axis=1)

# Show the updated DataFrame with the new column
delivery_data[['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'Distance_km']].head()


# EDA

In [ ]:
# Set up the visualization configuration
sns.set(style="whitegrid")

In [ ]:
# Summary statistics for numerical features
numerical_summary = delivery_data[['Delivery_person_Age', 'Delivery_person_Ratings', 'Distance_km', 'Time_taken(min)']].describe()


In [ ]:
# Plotting distributions of numerical features
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))
sns.histplot(delivery_data['Delivery_person_Age'], bins=20, kde=True, ax=axes[0, 0]).set_title('Distribution of Delivery Person Age')
sns.histplot(delivery_data['Delivery_person_Ratings'], bins=20, kde=True, ax=axes[0, 1]).set_title('Distribution of Delivery Person Ratings')
sns.histplot(delivery_data['Distance_km'], bins=20, kde=True, ax=axes[1, 0]).set_title('Distribution of Distance (km)')
sns.histplot(delivery_data['Time_taken(min)'], bins=20, kde=True, ax=axes[1, 1]).set_title('Distribution of Time Taken (min)')


Delivery_person_Age: The age ranges from 15 to 50 years with a mean of approximately 29.5 years. The distribution is fairly uniform with a slight skew towards younger ages.

Delivery_person_Ratings: Ratings are mostly high, clustered around 4.6 to 4.8, with an overall range from 1.0 to 6.0. Note that there are ratings above 5, which might indicate data entry errors or a different rating scale.

Distance_km: Most distances are relatively short, though there are extreme values (up to 19,692 km, which is unrealistic and likely indicates data issues). The median distance is around 9.26 km, suggesting most deliveries are local.

Time_taken(min): Delivery times range from 10 to 54 minutes, with an average delivery time of about 26 minutes. The distribution is fairly normal but slightly skewed right.


In [ ]:
numerical_summary

Potential Issues and Outliers:

There are apparent outliers in the Distance_km that could be due to data entry errors or incorrect geolocation data. These need to be addressed as they can significantly affect model accuracy.
The Delivery_person_Ratings column has values above the typical maximum of 5, suggesting either a different scale or erroneous entries.


# Data cleaning issues  

Correcting Ratings: Since ratings should logically be on a scale from 1 to 5, any ratings above 5 are likely erroneous and need to be addressed. We can set a strategy to either cap them at 5 or investigate these entries further to determine the correct action.
Handling Outliers in Distance: Extremely large values in the Distance_km seem unrealistic, especially for food deliveries. We'll set a reasonable maximum threshold for distances and investigate entries exceeding this threshold.


In [ ]:
# Capping ratings at 5.0
delivery_data['Delivery_person_Ratings'] = delivery_data['Delivery_person_Ratings'].apply(lambda x: min(x, 5.0))

# Analyze the distribution of Distance_km to decide on a threshold for capping
distance_percentiles = delivery_data['Distance_km'].quantile([0.90, 0.95, 0.99])

distance_percentiles


The 99th percentile for the distance is approximately 20.97 km. Given this analysis, a reasonable threshold to consider capping the distance might be around 21 km, which would help manage the extreme outliers without affecting the majority of the data.



In [ ]:
# Capping distance at 21 km
delivery_data['Distance_km'] = delivery_data['Distance_km'].apply(lambda x: min(x, 21.0))

# Plot the updated distribution of Distance_km
sns.histplot(delivery_data['Distance_km'], bins=20, kde=True).set_title('Updated Distribution of Distance (km)')
plt.show()


The updated distribution of Distance_km now reflects a more realistic range, with all distances capped at 21 km, addressing the issue of extreme outliers effectively.

# examining the categorical features Type_of_order and Type_of_vehicle in the dataset

In [ ]:
# Counting the frequency of each category in Type_of_order and Type_of_vehicle
order_type_counts = delivery_data['Type_of_order'].value_counts()
vehicle_type_counts = delivery_data['Type_of_vehicle'].value_counts()

# Plotting the distributions
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
sns.barplot(x=order_type_counts.index, y=order_type_counts.values, ax=axes[0]).set_title('Distribution of Type of Order')
sns.barplot(x=vehicle_type_counts.index, y=vehicle_type_counts.values, ax=axes[1]).set_title('Distribution of Type of Vehicle')

plt.tight_layout()
plt.show()

order_type_counts, vehicle_type_counts


# Categorical Feature Analysis:
Type of Order:

The orders are quite evenly distributed across four categories: Snack, Meal, Drinks, and Buffet.
Type of Vehicle:

The majority of deliveries are made using motorcycles, followed by scooters, and then electric scooters. Bicycles are very rarely used.
These distributions suggest that both order type and vehicle type are well-represented in the dataset, which allows for a comprehensive analysis of their impacts on delivery times.

# encode the categorical variables Type_of_order and Type_of_vehicle using one-hot encoding

In [ ]:
# One-hot encoding the categorical variables
categorical_encoded = pd.get_dummies(delivery_data[['Type_of_order', 'Type_of_vehicle']], drop_first=True)

# Joining the encoded columns back to the original DataFrame
delivery_data_encoded = delivery_data.join(categorical_encoded)

# Display the updated DataFrame with encoded categorical variables
delivery_data_encoded.head()


# correlation matrix

In [ ]:
# Exclude non-numeric columns and recompute the correlation matrix
numeric_data = delivery_data_encoded.select_dtypes(include=['float64', 'int64', 'uint8'])
correlation_matrix_numeric = numeric_data.corr()

# Visualizing the correlation matrix using a heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix_numeric, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix for Numeric Data')
plt.show()


# Feature Selection:

In [ ]:
from sklearn.model_selection import train_test_split

# Features selection
features = numeric_data.drop(columns=['Time_taken(min)'])  # Drop non-feature columns
target = numeric_data['Time_taken(min)']  # Target variable

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Display the shape of the train and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape


The data has been split into training and testing sets, with 36,474 observations for training and 9,119 for testing, across 13 features.

In [ ]:
# Model Building:

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate MAE and RMSE
mae = mean_absolute_error(y_test, y_pred)
rmse = sqrt(mean_squared_error(y_test, y_pred))

mae, rmse


Mean Absolute Error (MAE): 6.26 minutes
Root Mean Squared Error (RMSE): 8.03 minutes
These metrics indicate that, on average, the model's predictions are about 6.26 minutes off from the actual delivery times, and the RMSE gives a sense of the standard deviation of the prediction errors.

Interpretation:
The results suggest that the model has a reasonable level of accuracy but might still benefit from further tuning or possibly using additional features. The errors are not too high, which means the model can provide a decent baseline for predicting delivery times.

Model Tuning: Adjusting model parameters or trying more sophisticated algorithms to improve accuracy.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Reducing the training data size for quicker grid search (using 10% of the training data)
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)

# Simplified parameter grid
simplified_param_grid = {
    'n_estimators': [100, 150],  # Fewer options for number of trees
    'max_depth': [None, 20],  # Fewer depth options
    'min_samples_split': [2, 5]  # Fewer splitting options
}

# Create a simplified GridSearchCV object
simplified_grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                                      param_grid=simplified_param_grid, 
                                      cv=3,  # Keeping cross-validation
                                      scoring='neg_mean_squared_error',  # Using MSE for scoring
                                      verbose=1)

# Perform the simplified grid search
simplified_grid_search.fit(X_train_sample, y_train_sample)

# Best parameters and best score from the simplified grid search
simplified_best_params = simplified_grid_search.best_params_
simplified_best_score = sqrt(-simplified_grid_search.best_score_)  # Convert MSE to RMSE

simplified_best_params, simplified_best_score


Best Parameters:
n_estimators: 100 (Number of trees)
max_depth: None (No maximum depth)
min_samples_split: 5 (Minimum number of samples required to split a node)
Best RMSE: 7.98 minutes

In [ ]:
# Initialize the Random Forest Regressor with the best parameters from the grid search
optimized_rf_model = RandomForestRegressor(n_estimators=simplified_best_params['n_estimators'],
                                           max_depth=simplified_best_params['max_depth'],
                                           min_samples_split=simplified_best_params['min_samples_split'],
                                           random_state=42)

# Train the optimized model on the full training set
optimized_rf_model.fit(X_train, y_train)

# Predict on the test set
optimized_y_pred = optimized_rf_model.predict(X_test)

# Calculate MAE and RMSE for the optimized model
optimized_mae = mean_absolute_error(y_test, optimized_y_pred)
optimized_rmse = sqrt(mean_squared_error(y_test, optimized_y_pred))

optimized_mae, optimized_rmse


In [ ]:
# Adjust the model for quicker cross-validation by reducing the number of estimators
quick_rf_model = RandomForestRegressor(n_estimators=50, max_depth=simplified_best_params['max_depth'],
                                       min_samples_split=simplified_best_params['min_samples_split'], random_state=42)

# Performing 5-fold cross-validation with the adjusted model
quick_cv_scores = cross_val_score(quick_rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Calculate RMSE for each fold
quick_cv_rmse_scores = np.sqrt(-quick_cv_scores)

# Summary of cross-validation RMSE scores
quick_cv_rmse_mean = np.mean(quick_cv_rmse_scores)
quick_cv_rmse_std = np.std(quick_cv_rmse_scores)

quick_cv_rmse_scores, quick_cv_rmse_mean, quick_cv_rmse_std


In [ ]:
# Extract feature importances from the model
feature_importances = optimized_rf_model.feature_importances_

# Creating a DataFrame to view the features and their importances
feature_importance_df = pd.DataFrame({
    'Feature': features.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Visualizing the feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
plt.title('Feature Importances in Random Forest Model')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

feature_importance_df


Delivery_person_Ratings: This feature has the highest importance, indicating that the ratings of a delivery person are a strong predictor of delivery times.
Distance_km: As expected, the distance between the restaurant and the delivery location is also a significant predictor.
Delivery_person_Age: The age of the delivery person shows moderate importance, suggesting it has some but not a dominant influence on delivery times.
Geographical Coordinates: The longitude and latitude for both the delivery location and the restaurant have varying degrees of importance, likely reflecting the geographical factors that affect delivery times, such as urban density or traffic patterns.
These results can guide business decisions and model refinement. For instance, the high importance of delivery person ratings could suggest investing in training or incentives for delivery personnel to maintain high service quality, which apparently correlates with faster delivery.

In [ ]:
# Assuming you have already loaded your dataset and split it into training and testing sets
# and your model 'optimized_rf_model' is already trained

# Make predictions on the test set
y_pred = optimized_rf_model.predict(X_test)

# Calculate residuals (differences between actual and predicted values)
residuals = y_test - y_pred

# Plotting the residuals
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.title('Residual Plot')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.axhline(y=0, color='red', linestyle='--')  # A line at zero which ideally should have all points
plt.show()
